In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC50_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 2
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC50(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC50(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC50(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料
讀取3秒資料
讀取5秒資料
Data Loading Time: 0.25 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []


#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    print("----------------------------------------------------------------------------------------------------------------")
    print("fold: ", fold)
    print("----------------------------------------------------------------------------------------------------------------")

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=50)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    #------------------------------------------------
    #需要調整的地方
    #------------------------------------------------
    #學習率
    lr= 0.00032
    #epoch
    epoch= 120
    #訓練資料(要用幾秒訓練) data_1S/data_3S/data_5S
    trainData = data_5S
    #------------------------------------------------
    
    #紀錄訓練開始時間
    start_time = time.time()

    #訓練模型
    model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold,
                                                                setLR=lr,
                                                                setEpoch=epoch)
    
    #測試準確度
    test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
    test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
    test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

    #紀錄訓練結束時間
    end_time = time.time()

    #計算訓練耗時
    training_time = end_time - start_time

    #印出訓練耗時
    print("Training Time: {:.2f} seconds".format(training_time))

    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("訓練過程\n")
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    for i in range(len(train_loss)):
        # 寫入資料
        f.write("Fold: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
    f.write("Training Time: {:.2f} seconds\n".format(training_time))
    f.write("---------------------------------------------------------------------------------------------------------------\n")

    #--------------------------------------------------------------------

    
    kFoldTrainTime.append(training_time)
    kFoldTrainAccuracy.append(train_accuracy[-1])
    kFoldTestAccuracy1.append(test_accuracy1)
    kFoldTestAccuracy3.append(test_accuracy3)
    kFoldTestAccuracy5.append(test_accuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

----------------------------------------------------------------------------------------------------------------
fold:  1
----------------------------------------------------------------------------------------------------------------
可用的 GPU 数量: 1
Train Epoch: 1/120 Train_Loss: 2.708496332168579 Train_acc: 18.562500%
Train Epoch: 2/120 Train_Loss: 1.9574155807495117 Train_acc: 45.000000%
Train Epoch: 3/120 Train_Loss: 1.2188305854797363 Train_acc: 62.625000%
Train Epoch: 4/120 Train_Loss: 0.9605581164360046 Train_acc: 78.062500%
Train Epoch: 5/120 Train_Loss: 0.6148308515548706 Train_acc: 87.125000%
Train Epoch: 6/120 Train_Loss: 0.33989953994750977 Train_acc: 93.125000%
Train Epoch: 7/120 Train_Loss: 0.19762036204338074 Train_acc: 96.750000%
Train Epoch: 8/120 Train_Loss: 0.12974359095096588 Train_acc: 98.437500%
Train Epoch: 9/120 Train_Loss: 0.09054747223854065 Train_acc: 98.125000%
Train Epoch: 10/120 Train_Loss: 0.16716043651103973 Train_acc: 98.062500%
Train Epoch: 11/120 Train_

Train Epoch: 106/120 Train_Loss: 0.0026031159795820713 Train_acc: 100.000000%
Train Epoch: 107/120 Train_Loss: 0.0017953547649085522 Train_acc: 100.000000%
Train Epoch: 108/120 Train_Loss: 0.0033646011725068092 Train_acc: 100.000000%
Train Epoch: 109/120 Train_Loss: 0.0027697524055838585 Train_acc: 100.000000%
Train Epoch: 110/120 Train_Loss: 0.0019375457195565104 Train_acc: 100.000000%
Train Epoch: 111/120 Train_Loss: 0.0022347737103700638 Train_acc: 100.000000%
Train Epoch: 112/120 Train_Loss: 0.0022180797532200813 Train_acc: 100.000000%
Train Epoch: 113/120 Train_Loss: 0.002290130592882633 Train_acc: 100.000000%
Train Epoch: 114/120 Train_Loss: 0.0025281780399382114 Train_acc: 100.000000%
Train Epoch: 115/120 Train_Loss: 0.0032038057688623667 Train_acc: 100.000000%
Train Epoch: 116/120 Train_Loss: 0.0014852514723315835 Train_acc: 100.000000%
Train Epoch: 117/120 Train_Loss: 0.001997586339712143 Train_acc: 100.000000%
Train Epoch: 118/120 Train_Loss: 0.0015333292540162802 Train_acc: 

Train Epoch: 89/120 Train_Loss: 0.0015622256323695183 Train_acc: 100.000000%
Train Epoch: 90/120 Train_Loss: 0.0011427553836256266 Train_acc: 100.000000%
Train Epoch: 91/120 Train_Loss: 0.0012069575022906065 Train_acc: 100.000000%
Train Epoch: 92/120 Train_Loss: 0.0009406215394847095 Train_acc: 100.000000%
Train Epoch: 93/120 Train_Loss: 0.0010367508511990309 Train_acc: 100.000000%
Train Epoch: 94/120 Train_Loss: 0.001543726189993322 Train_acc: 100.000000%
Train Epoch: 95/120 Train_Loss: 0.0009599598124623299 Train_acc: 100.000000%
Train Epoch: 96/120 Train_Loss: 0.002184940269216895 Train_acc: 100.000000%
Train Epoch: 97/120 Train_Loss: 0.0009438231936655939 Train_acc: 100.000000%
Train Epoch: 98/120 Train_Loss: 0.0018149992683902383 Train_acc: 100.000000%
Train Epoch: 99/120 Train_Loss: 0.0012760787503793836 Train_acc: 100.000000%
Train Epoch: 100/120 Train_Loss: 0.0008350316202268004 Train_acc: 100.000000%
Train Epoch: 101/120 Train_Loss: 0.0009336440707556903 Train_acc: 100.000000%

Train Epoch: 72/120 Train_Loss: 0.0024473348166793585 Train_acc: 100.000000%
Train Epoch: 73/120 Train_Loss: 0.0023905783891677856 Train_acc: 100.000000%
Train Epoch: 74/120 Train_Loss: 0.0036730095744132996 Train_acc: 100.000000%
Train Epoch: 75/120 Train_Loss: 0.0019645290449261665 Train_acc: 100.000000%
Train Epoch: 76/120 Train_Loss: 0.002328480128198862 Train_acc: 100.000000%
Train Epoch: 77/120 Train_Loss: 0.0019471719861030579 Train_acc: 100.000000%
Train Epoch: 78/120 Train_Loss: 0.0020718637388199568 Train_acc: 100.000000%
Train Epoch: 79/120 Train_Loss: 0.002509552985429764 Train_acc: 100.000000%
Train Epoch: 80/120 Train_Loss: 0.0029022344388067722 Train_acc: 100.000000%
Train Epoch: 81/120 Train_Loss: 0.00246371328830719 Train_acc: 100.000000%
Train Epoch: 82/120 Train_Loss: 0.0021364602725952864 Train_acc: 100.000000%
Train Epoch: 83/120 Train_Loss: 0.0024539404548704624 Train_acc: 100.000000%
Train Epoch: 84/120 Train_Loss: 0.002638982841745019 Train_acc: 100.000000%
Trai

Train Epoch: 55/120 Train_Loss: 0.0014494479401037097 Train_acc: 100.000000%
Train Epoch: 56/120 Train_Loss: 0.0011525355512276292 Train_acc: 100.000000%
Train Epoch: 57/120 Train_Loss: 0.001425122027285397 Train_acc: 100.000000%
Train Epoch: 58/120 Train_Loss: 0.0019995777402073145 Train_acc: 100.000000%
Train Epoch: 59/120 Train_Loss: 0.0013109002029523253 Train_acc: 100.000000%
Train Epoch: 60/120 Train_Loss: 0.0019281089771538973 Train_acc: 100.000000%
Train Epoch: 61/120 Train_Loss: 0.0009044401813298464 Train_acc: 100.000000%
Train Epoch: 62/120 Train_Loss: 0.0013991112355142832 Train_acc: 100.000000%
Train Epoch: 63/120 Train_Loss: 0.001614194829016924 Train_acc: 100.000000%
Train Epoch: 64/120 Train_Loss: 0.0015984858619049191 Train_acc: 100.000000%
Train Epoch: 65/120 Train_Loss: 0.001155361533164978 Train_acc: 100.000000%
Train Epoch: 66/120 Train_Loss: 0.0010264635784551501 Train_acc: 100.000000%
Train Epoch: 67/120 Train_Loss: 0.0015878487611189485 Train_acc: 100.000000%
Tr

Train Epoch: 38/120 Train_Loss: 0.0021461353171616793 Train_acc: 100.000000%
Train Epoch: 39/120 Train_Loss: 0.0016031116247177124 Train_acc: 100.000000%
Train Epoch: 40/120 Train_Loss: 0.0019893296994268894 Train_acc: 100.000000%
Train Epoch: 41/120 Train_Loss: 0.002006695605814457 Train_acc: 100.000000%
Train Epoch: 42/120 Train_Loss: 0.0015892130322754383 Train_acc: 100.000000%
Train Epoch: 43/120 Train_Loss: 0.0024240564089268446 Train_acc: 100.000000%
Train Epoch: 44/120 Train_Loss: 0.0012694537872448564 Train_acc: 100.000000%
Train Epoch: 45/120 Train_Loss: 0.0017102209385484457 Train_acc: 100.000000%
Train Epoch: 46/120 Train_Loss: 0.002048724563792348 Train_acc: 100.000000%
Train Epoch: 47/120 Train_Loss: 0.0013875211589038372 Train_acc: 100.000000%
Train Epoch: 48/120 Train_Loss: 0.0018148546805605292 Train_acc: 100.000000%
Train Epoch: 49/120 Train_Loss: 0.0018307940335944295 Train_acc: 100.000000%
Train Epoch: 50/120 Train_Loss: 0.0015763473929837346 Train_acc: 100.000000%
T

In [5]:
#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    # 寫入資料
    f.write("Fold: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, kFoldTrainAccuracy[fold], kFoldTestAccuracy1[fold], kFoldTestAccuracy3[fold], kFoldTestAccuracy5[fold], kFoldTrainTime[fold]))
    f.write("-----------------------------------------------------------------------------------\n")
# 寫入資料
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(max(kFoldTrainAccuracy), max(kFoldTestAccuracy1), max(kFoldTestAccuracy3), max(kFoldTestAccuracy5)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(min(kFoldTrainAccuracy), min(kFoldTestAccuracy1), min(kFoldTestAccuracy3), min(kFoldTestAccuracy5)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(np.mean(kFoldTrainAccuracy), np.mean(kFoldTestAccuracy1), np.mean(kFoldTestAccuracy3), np.mean(kFoldTestAccuracy5)))

f.close()